In [1]:
import torch

from prime_model import device
from population import Population

In [2]:
# 20 0 or 1 bits to represent the number
inputs = 20
# 2 possible outputs: prime or not prime
outputs = 2

In [3]:
def gen_primes(n):
    sieve = [True] * n
    for i in range(2, int(n ** 0.5) + 1):
        if sieve[i]:
            for j in range(i*i, n, i):
                sieve[j] = False
    sieve[0] = False
    sieve[1] = False
    return sieve

upper_bound = 2 ** inputs - 1
primes = gen_primes(upper_bound)

In [4]:
import random

size = 250_000
dataset_numbers = random.sample(range(3, upper_bound, 2), size)

train_size = int(size * 0.8)
train_numbers = dataset_numbers[:train_size]
test_numbers = dataset_numbers[train_size:]

In [5]:
x_train = torch.tensor([list(map(int, [*format(i, "020b")]))
                        for i in train_numbers], dtype=torch.float, device=device)
x_test = torch.tensor([list(map(int, [*format(i, "020b")]))
                       for i in test_numbers], dtype=torch.float, device=device)
y_train = torch.tensor([[0, 1] if primes[i] else [1, 0]
                       for i in train_numbers], dtype=torch.float, device=device)
y_test = torch.tensor([[0, 1] if primes[i] else [1, 0]
                       for i in test_numbers], dtype=torch.float, device=device)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

torch.Size([200000, 20]) torch.Size([200000, 2])
torch.Size([50000, 20]) torch.Size([50000, 2])


In [6]:
population = Population()
best_agent = population.simulate(x_train, y_train, mutation_rate=0.01)

/home/hlias/.local/lib/python3.11/site-packages/torch/nn/modules/container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Generation 10 Best loss: 1.1241936683654785
Generation 20 Best loss: 1.1239060163497925
Generation 30 Best loss: 1.1236482858657837
Generation 40 Best loss: 1.1233091354370117
Generation 50 Best loss: 1.1231203079223633
Generation 60 Best loss: 1.1229485273361206
Generation 70 Best loss: 1.1227960586547852
Generation 80 Best loss: 1.1226543188095093
Generation 90 Best loss: 1.1225756406784058
Generation 100 Best loss: 1.1225216388702393


In [7]:
general_accuracy = 0
primes_accuracy = 0
primes_count = 0
for x, y in zip(x_test, y_test):
    prediction = best_agent.forward(x)
    if y.argmax() == 1:
        primes_count += 1
    if prediction.argmax() == y.argmax():
        general_accuracy += 1
        if y.argmax() == 1:
            primes_accuracy += 1

print(f"General accuracy: {general_accuracy / len(x_test) * 100:.2f}%")
print(f"Primes accuracy: {primes_accuracy / primes_count * 100:.2f}% from {primes_count} primes")

General accuracy: 72.19%
Primes accuracy: 21.59% from 7919 primes
